In [1]:
%run stdPackages.ipynb # this imports a lot of useful packages

# The ```modelShell``` class

The ```modelShell``` class is build on top of the ```lpCompiler.lpBlock``` class (see documentation [**Part I**](./lpCompiler_PartI.ipynb) and [**Part II**](./lpCompiler_PartII.ipynb)). It is used as the parent class for all of the linear models used throughout the course. The core functionality is that it establishes a routine of: 1) Perform some to-be-specified 'presolve' routines (```self.preSolve```), 2) set up ```lpBlock``` with a to-be-specified structure, and 3) read and structure the model solution in terms of the same structure as the ```lpBlock```. Specifically, we can write a custom ```postSolve``` routine to go through.

The note is structured as follows:

>[**1. Unload Solution**](#unloadToDb): Shows how we can use the ```lpBlock``` model structure to automatically report the solution in terms of variable/constraint names and pandas indices.  
>[**2. Create model with block structure**](#modelStructure): Shows an example of how to incorporate the model structure from ```lpBlock``` in the ```modelShell```.  
>[**3. Create pre- and post solve routines**](#modelRoutines): The 

## 1. Unload solution to database <a id='unloadToDb'></a>

We borrow the model structure and database from the documentation of ```lpBlock```:

In [2]:
with open(os.path.join(d['data'],'blockPartI'), "rb") as file:
    block = pickle.load(file)
with open(os.path.join(d['data'],'blockPartI_db'), "rb") as file:
    db = pickle.load(file)

Recall that we can solve this model simply by running:

In [3]:
sol = optimize.linprog(**block())
sol

        message: Optimization terminated successfully. (HiGHS Status 7: Optimal)
        success: True
         status: 0
            fun: -12.5
              x: [-0.000e+00  0.000e+00  5.000e-01  5.000e-01  0.000e+00
                   5.000e-01  2.500e-01  2.500e-01]
            nit: 2
          lower:  residual: [-0.000e+00  0.000e+00  5.000e-01  5.000e-01
                              0.000e+00  5.000e-01  2.500e-01  2.500e-01]
                 marginals: [ 0.000e+00  5.000e+00  0.000e+00  0.000e+00
                              5.000e+00  0.000e+00  0.000e+00  0.000e+00]
          upper:  residual: [ 5.000e-01  5.000e-01  0.000e+00  0.000e+00
                              5.000e-01  0.000e+00  2.500e-01  0.000e+00]
                 marginals: [ 0.000e+00  0.000e+00 -1.000e+01 -5.000e+00
                              0.000e+00 -5.000e+00  0.000e+00 -1.000e+01]
          eqlin:  residual: [ 0.000e+00  0.000e+00]
                 marginals: [ 1.000e+01  1.500e+01]
        ineqlin:  r

The method ```unloadToDb(self, sol)``` from ```modelShell``` uses the model structure from ```lpBLock``` to restructure the solution in our database. To access this, let us initialize a model with the structure from 'block' and unload the solution to the database:

In [4]:
model = lpModels.modelShell(db, blocks = block)
model.unloadToDb(sol)

This initializes the model and adds the model structure to ```self.blocks```:

In [5]:
model.blocks == block

True

First, note that instead of the solution vector $\mathbf{x}$, the solution is now stored using their variable names $D$ and $E$:

In [6]:
model.db['D']

c           h
Consumer 1  1   -0.0
            2    0.0
Consumer 2  1    0.5
            2    0.5
dtype: float64

Second, note that we also automatically retrieve the shadow values on the constraints of the problem. These are stored with the syntax "$\lambda$\_z" for the constraint "z". In our case, we had the equilibrium constraint:

In [7]:
model.db['λ_equilibrium']

h_constr  _type
1         eq       10.0
2         eq       15.0
dtype: float64

The ```_type``` index indicates what type of constraint we are talking about; in this case, an equality constraint. In our case, we also have domain constraints on our two variables $D$, $E$. The shadow values of these domain constraints are also automatically stored with the syntax "$\lambda$\_z" for the variable "z":

In [8]:
model.db['λ_D']

c           h  _type
Consumer 1  1  l         0.0
            2  l         5.0
Consumer 2  1  l         0.0
            2  l         0.0
Consumer 1  1  u         0.0
            2  u         0.0
Consumer 2  1  u       -10.0
            2  u        -5.0
dtype: float64

The ```_type``` index indicates whether this is the lower (```l```) or the upper (```u```) bound. 

## 2. Create model with block structure <a id='modelStructure'></a>

In the example above, we used the ```modelShell``` simply to unload the solution from an ```lpBlock``` in a structured way. However, usually, we will define the ```modelShell``` with an internal model structure. We do this by adding a ```initBlocks``` method to the class. There is not a single way to this, but rather a host of ways. Here, we use a method similar to the ones used throughout the course.

We borrow from the code summary in the [documentation to](./lpCompiler_PartI.ipynb) ```lpCompiler```. Specifically, we define a new model class that uses block structure from previous:

In [9]:
_blocks = ('c','l','u','b_eq','b_ub','A_eq','A_ub')
class customModel(lpModels.modelShell):
    def __init__(self, db, **kwargs):
        super().__init__(db, **kwargs) # initialize like the parent class ```modelShell```
    @property
    def globalDomains(self):
        return {'E': pd.MultiIndex.from_product([self.db['id'], self.db['h']]),
                'D': pd.MultiIndex.from_product([self.db['c'],  self.db['h']]),
                'equilibrium':  self.db['h_constr']}
    @property
    def c(self):
        return [{'varName': 'E', 'value' : pyDbs.broadcast(self.db['mc'], self.globalDomains['E'])},
                {'varName': 'D', 'value' : -pyDbs.broadcast(self.db['MWP'], self.globalDomains['D'])}]
    @property
    def u(self):
        return [{'varName': 'E', 'value' : self.db['q']},
                {'varName': 'D', 'value' : pyDbs.broadcast(self.db['D̄'], self.globalDomains['D'])}]
    @property
    def b_eq(self):
        return [{'constrName': 'equilibrium'}]
    @property
    def A_eq(self):
        return [{'constrName': 'equilibrium', 'varName': 'E', 'value': base.appIndexWithCopySeries(pd.Series(1, index = self.globalDomains['E']), 'h','h_constr')},
                {'constrName': 'equilibrium', 'varName': 'D', 'value': base.appIndexWithCopySeries(pd.Series(-1, index = self.globalDomains['D']), 'h','h_constr')}]
    
    def initBlocks(self, **kwargs):
        [getattr(self.blocks, f'add_{t}')(**v) for t in _blocks if hasattr(self,t) for v in getattr(self,t)];

This class, ```customModel```, creates the model structure that we otherwise loaded from ```block```, by calling the method ```initBlocks```:

In [10]:
model = customModel(db)
model.initBlocks()

## 3. Create pre- and post solve routines <a id='modelRoutines'></a>

Another way to use the ```modelShell``` class is to use the pre-specified ```__call__``` method: As was the case with the ```lpBlock``` class, this method runs through some "standard" steps:
1. ```preSolve``` routine (if one is specified),
2. ```initBlocks```, if such a method is specified; otherwise, the existing ```self.blocks``` attribute is used as model structure).
3. ```solve```: solves the ```self.blocks``` model and runs through a ```postSolve``` routine. The default ```postSolve``` routine is simply to unload the solution to the database (cf. section 1).

In our case with the ```customModel``` specified above, relying on the ```__call__``` method automatically sets up the model structure, solves the model, and unloads the solution to the database. All this by calling:

In [11]:
model()

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


We can specify our own ```preSolve``` and ```postSolve``` methods if there is a standard routine that we want the model to perform. This can be useful, it there are simple equations included in our model that we do not necessarily need in the optimization proces. For instance, say that we had data on the fuels that each of the generators used. 

Specifically, let $\mu_{id, f}$ denote GJ of fuel type $f$ that generator $id$ needs to produce 1 GJ of electricity. Similarly, let $\phi_f$ denote the CO2 intensity for fuel $f$ (i.e. ton of CO2 from 1 GJ of fuel $f$). In this case, we can define the fuel use $F_f$ and total emissions $M$ from the equations:

$$\begin{align}
    F_f =& \sum_{h, id} \mu_{id,f} \cdot E_{id,h} \\ 
    M   =& \sum_f \phi_f \cdot F_f.
\end{align}$$

To see how we include these equations in the model, let us define a small example that is consistent with our current model formulation with two generators (conv. plant and wind turbine):

In [12]:
model.db['μ'] = pd.Series([1.25, 0], index = pd.MultiIndex.from_tuples([('Conv. plant', 'Coal'), ('Wind turbine', 'Coal')], names = ['id','f']))
model.db['ϕ'] = pd.Series([0.09], index = pd.Index(['Coal'], name = 'f'))

Given the model solution, note that we can simply solve for $F_f$ and $M$ as follows:

In [21]:
F = (model.db['E'] * model.db['μ']).groupby('f').sum()
M = sum(model.db['ϕ'] * F)
F

f
Coal    0.3125
dtype: float64

We can make sure that we always report this by adding it to the ```postSolve``` routine. Specifically:

In [14]:
def postSolve(solution, **kwargs):
    if solution['status'] == 0: # if optimization is successfull 
        model.unloadToDb(solution) # standard unload solution
        model.db['F'] = (model.db['E'] * model.db['μ']).groupby('f').sum()
        model.db['M'] = sum(model.db['ϕ'] * model.db['F'])
model.postSolve = postSolve

Now, if we run the standard ```__call__``` method, this automatically computes the fuel and emissions variables

In [20]:
model()
model.db['F']

Solution status 0: Optimization terminated successfully. (HiGHS Status 7: Optimal)


f
Coal    0.3125
dtype: float64